In [1]:
import xml.etree.cElementTree as ET
import pandas as pd
import time, os
import concurrent.futures as cf

In [5]:

data_list = [
    
    'US_XML_AddFeed_20190319_20190325.xml', 'US_XML_AddFeed_20190305_20190311.xml',
    'US_XML_AddFeed_20190326_20190401.xml', 'US_XML_AddFeed_20190402_20190408.xml'
    
]

In [ ]:
%%time

def parse_xml(file):

    root = ET.parse(file + '.xml').getroot()

    start_time = time.time()
    counter = 0
    jobs_list = []
    for node in root:
        job_dict = {}
        for elem in node:
            # print(elem.tag)
            # print(elem.text)
            if elem.tag not in job_dict.keys():
                if elem.tag == "CanonSkills":
                    skill_dict = {}
                    for skill in elem:
                        if skill.attrib:
                            skill_dict.update({skill.attrib['name']: skill.attrib['clusterName']})

                    job_dict[elem.tag] = skill_dict

                else:
                    job_dict[elem.tag] = elem.text
            else:
                print(elem.tag)
        jobs_list.append(job_dict)
        counter += 1
        if counter % 1000 == 0:
            elapsed_time = time.time() - start_time
            print(str(counter) + " records done, elapsed time: "+str(elapsed_time)+" sec")

    out_df = pd.DataFrame(jobs_list)

    out_df.to_csv(f'{file}.csv', index=False)
    
    
parse_xml(data_list)

Before running the function on your list of data make sure you run the `os.cpu_count()` function below to determine how many cores you have in your machine. XML is not fun to parse and it takes a while, so I would recommmend to leave a few cores available so that you can use your computer while the code is running, if you have a spare computer available, then remove the `max_workers` parameter below to use all cores.

In [2]:
os.cpu_count()

8

In [ ]:
%%time

with cf.ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(parse_xml, data_list)